In [1]:
import angr
proj = angr.Project("/bin/true")

WARNING | 2021-03-22 00:21:21,992 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


## basic properties

In [21]:
def bin_info(proj):
    print(f"[file::{proj.filename}]")
    print(f"proj.arch = {proj.arch}")
    print(f"\tbits  = {proj.arch.bits}")
    print(f"\tbytes = {proj.arch.bytes}")
    print(f"proj.entry = {hex(proj.entry)}")
    
    # loader information
    
    print()
    print(f"[main_object::{proj.loader.main_object}")
    print(f"execstack? {proj.loader.main_object.execstack}")
    print(f"pie?       {proj.loader.main_object.pic}")

In [22]:
bin_info(proj)

[file::/bin/true]
proj.arch = <Arch AMD64 (LE)>
	bits  = 64
	bytes = 8
proj.entry = 0x4023c0

[main_object::<ELF Object true, maps [0x400000:0x40a377]>
execstack? False
pie?       True


## the loader

In [18]:
print(proj.loader)
for k, v in proj.loader.shared_objects.items():
    print(f"{k: <25}{v}")
print(hex(proj.loader.min_addr))
print(hex(proj.loader.max_addr))

<Loaded true, maps [0x400000:0xa07fff]>
true                     <ELF Object true, maps [0x400000:0x40a377]>
libc.so.6                <ELF Object libc-2.31.so, maps [0x500000:0x6c4507]>
ld-linux-x86-64.so.2     <ELF Object ld-2.31.so, maps [0x700000:0x72c177]>
extern-address space     <ExternObject Object cle##externs, maps [0x800000:0x87ffff]>
cle##tls                 <ELFTLSObjectV2 Object cle##tls, maps [0x900000:0x91500f]>
0x400000
0xa07fff
<ELF Object true, maps [0x400000:0x40a377]>


## the factory

### blocks

In [23]:
block = proj.factory.block(proj.entry)

In [25]:
block.pp()

0x4023c0:	xor	ebp, ebp
0x4023c2:	mov	r9, rdx
0x4023c5:	pop	rsi
0x4023c6:	mov	rdx, rsp
0x4023c9:	and	rsp, 0xfffffffffffffff0
0x4023cd:	push	rax
0x4023ce:	push	rsp
0x4023cf:	lea	r8, [rip + 0x34ba]
0x4023d6:	lea	rcx, [rip + 0x3453]
0x4023dd:	lea	rdi, [rip - 0xe4]
0x4023e4:	call	qword ptr [rip + 0x7bf6]


In [26]:
block.instructions

11

### states

In [27]:
state = proj.factory.entry_state()

In [30]:
state.regs.rip

<BV64 0x4023c0>

In [31]:
state.regs.rax

<BV64 0x1c>

In [34]:
state.mem[proj.entry].int.resolved

<BV32 0x8949ed31>

In [36]:
bv = state.solver.BVV(0x1234, 32)
bv

<BV32 0x1234>

In [37]:
state.solver.eval(bv)

4660

In [40]:
state.regs.rsi = state.solver.BVV(3, 64)
state.regs.rsi

<BV64 0x3>

In [42]:
state.mem[0x1000].long = 4
state.mem[0x1000].long.resolved

<BV64 0x4>

### simulation managers

In [46]:
simgr = proj.factory.simulation_manager(state)

In [47]:
simgr

<SimulationManager with 1 active>

In [51]:
simgr.active

[<SimState @ 0x4023c0>]

In [52]:
simgr.step()

<SimulationManager with 1 active>

In [54]:
simgr.active

[<SimState @ 0x526c20>]

In [55]:
simgr.active[0].regs.rip

<BV64 0x526c20>

In [56]:
state.regs.rip

<BV64 0x4023c0>

## analyses

In [57]:
# constructing a control flow graph
proj = angr.Project("/bin/true", auto_load_libs=False)
cfg = proj.analyses.CFGFast()

WARNING | 2021-03-22 00:48:58,798 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.
WARNING | 2021-03-22 00:48:58,807 | angr.analyses.cfg.cfg_base | _is_region_extremely_sparse: The given region 0x4058b0-0x4058b9 is not a continuous memory region in the memory space. Only the first 9 bytes (0x4058b0-0x4058b8) are processed.


In [59]:
cfg.graph

In [60]:
len(cfg.graph.nodes())

1085

In [63]:
entry_node = cfg.model.get_any_node(proj.entry)

In [66]:
len(list(cfg.graph.successors(entry_node)))

1